## Churn Data — Dtype Optimization & Cleaning

### By:
jdg

### Date:
2026-02-21

### Description:

Loads the intermediate Parquet file produced by `1-data/01_jdg_churn_data_loading_20260221.ipynb`.
The goals of this notebook are:
- Clean dirty string values (rogue numeric strings injected into categorical columns)
- Fix `TotalCharges` dtype (object → float64)
- Cap extreme outliers at the 99th percentile: `MonthlyCharges` and `TotalCharges`
- Clip negative `TotalCharges` values to 0 (data corruption)
- Cast every column to its optimal dtype (bool, category, Int16, float64)
- Persist the result to `03_primary` as `churn_primary.parquet`

## 📚 Import libraries

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype

## 💾 Load data

In [3]:
INTERMEDIATE_PATH = Path("../../data/02_intermediate/Churn/churn_raw.parquet")

df = pd.read_parquet(INTERMEDIATE_PATH)

print(f"Loaded: {df.shape[0]:,} rows x {df.shape[1]} columns")
df.head()

Loaded: 14,214 rows x 20 columns


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0.0,Yes,No,1.0,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0.0,No,No,34.0,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0.0,No,No,2.0,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0.0,No,No,45.0,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0.0,No,No,2.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## 🔧 Cleaning & dtype optimization

### Step 1: Replace dirty string values → NaN

Several categorical columns contain a single rogue numeric string that was injected
into otherwise clean categorical data. Replace each with `np.nan` before casting.

In [4]:
dirty = {
    "MultipleLines": "1244132",
    "OnlineSecurity": "23453432",
    "DeviceProtection": "1243524",
    "StreamingTV": "5412335",
    "StreamingMovies": "1523434",
}
for col, val in dirty.items():
    df[col] = df[col].replace(val, np.nan)

print("Dirty values replaced with NaN.")
for col in dirty:
    print(f"  {col}: {df[col].isna().sum()} NaNs")

Dirty values replaced with NaN.
  MultipleLines: 138 NaNs
  OnlineSecurity: 191 NaNs
  DeviceProtection: 195 NaNs
  StreamingTV: 200 NaNs
  StreamingMovies: 179 NaNs


### Step 2: Fix `TotalCharges` — object → float64

`TotalCharges` was stored as object because some rows contain whitespace instead of a number.
`pd.to_numeric(..., errors='coerce')` converts those to `NaN`.

In [5]:
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

print(f"NaN in TotalCharges after cast: {df['TotalCharges'].isna().sum()}")

NaN in TotalCharges after cast: 93


### Step 3: Cap `MonthlyCharges` outlier at 99th percentile

The descriptive statistics revealed an extreme outlier (~5 × 10¹²) in `MonthlyCharges`.
Clip values above the 99th percentile to remove the influence of this single bad row.

In [6]:
cap = df["MonthlyCharges"].quantile(0.99)
df["MonthlyCharges"] = df["MonthlyCharges"].clip(upper=cap)

print(f"MonthlyCharges 99th-percentile cap: {cap:.2f}")
print(f"MonthlyCharges max after capping  : {df['MonthlyCharges'].max():.2f}")

MonthlyCharges 99th-percentile cap: 114.85
MonthlyCharges max after capping  : 114.85


### Step 3b: Cap `TotalCharges` at [0, 99th percentile]

`TotalCharges` contains negative values (data corruption) and extreme positives; clip to
[0, 99th percentile] matching `MonthlyCharges` treatment.

In [7]:
cap_tc = df["TotalCharges"].quantile(0.99)
df["TotalCharges"] = df["TotalCharges"].clip(lower=0, upper=cap_tc)

print(f"TotalCharges 99th-percentile cap: {cap_tc:.2f}")
print(f"TotalCharges max after capping  : {df['TotalCharges'].max():.2f}")

TotalCharges 99th-percentile cap: 8045.81
TotalCharges max after capping  : 8045.81


### Step 4: Cast Yes/No columns → bool

Columns with exactly two values (`Yes` / `No`) are mapped to Python `bool`.

In [8]:
yes_no_bool = ["Partner", "Dependents", "PhoneService", "PaperlessBilling", "Churn"]
for col in yes_no_bool:
    df[col] = df[col].map({"Yes": True, "No": False}).astype("boolean")

print("Yes/No columns cast to bool:")
for col in yes_no_bool:
    print(f"  {col}: {df[col].dtype}")

Yes/No columns cast to bool:
  Partner: boolean
  Dependents: boolean
  PhoneService: boolean
  PaperlessBilling: boolean
  Churn: boolean


### Step 5: Cast `SeniorCitizen` (0/1 float) → nullable boolean

In [9]:
df["SeniorCitizen"] = df["SeniorCitizen"].astype("boolean")

print(f"SeniorCitizen dtype: {df['SeniorCitizen'].dtype}")

SeniorCitizen dtype: boolean


### Step 6: Cast `gender` → unordered category

In [10]:
df["gender"] = df["gender"].astype("category")

print(f"gender dtype    : {df['gender'].dtype}")
print(f"gender categories: {df['gender'].cat.categories.tolist()}")

gender dtype    : category
gender categories: ['Female', 'Male']


### Step 7: Cast ternary service columns → unordered category

In [11]:
ternary_cols = [
    "MultipleLines",
    "InternetService",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
]
for col in ternary_cols:
    df[col] = df[col].astype("category")

print("Ternary service columns cast to category:")
for col in ternary_cols:
    print(f"  {col}: {df[col].cat.categories.tolist()}")

Ternary service columns cast to category:
  MultipleLines: ['No', 'No phone service', 'Yes']
  InternetService: ['DSL', 'Fiber optic', 'No']
  OnlineSecurity: ['No', 'No internet service', 'Yes']
  OnlineBackup: ['No', 'No internet service', 'Yes']
  DeviceProtection: ['No', 'No internet service', 'Yes']
  TechSupport: ['No', 'No internet service', 'Yes']
  StreamingTV: ['No', 'No internet service', 'Yes']
  StreamingMovies: ['No', 'No internet service', 'Yes']


### Step 8: Cast `PaymentMethod` → unordered category

In [12]:
df["PaymentMethod"] = df["PaymentMethod"].astype("category")

print(f"PaymentMethod categories: {df['PaymentMethod'].cat.categories.tolist()}")

PaymentMethod categories: ['Bank transfer (automatic)', 'Credit card (automatic)', 'Electronic check', 'Mailed check']


### Step 9: Cast `Contract` → ordered category

Contract length has a natural ordering: Month-to-month < One year < Two year.

In [13]:
contract_order = CategoricalDtype(
    categories=["Month-to-month", "One year", "Two year"], ordered=True
)
df["Contract"] = df["Contract"].astype(contract_order)

print(f"Contract dtype  : {df['Contract'].dtype}")
print(f"Contract ordered: {df['Contract'].cat.ordered}")
print(f"Contract cats   : {df['Contract'].cat.categories.tolist()}")

Contract dtype  : category
Contract ordered: True
Contract cats   : ['Month-to-month', 'One year', 'Two year']


### Step 10: Cast `tenure` → nullable Int16

In [14]:
df["tenure"] = df["tenure"].astype("Int16")

print(f"tenure dtype: {df['tenure'].dtype}")

tenure dtype: Int16


## ✅ Verification

In [15]:
df.info()
print()
print(f"object columns remaining: {(df.dtypes == 'object').sum()}")
print(f"Contract ordered        : {df['Contract'].cat.ordered}")
print(f"MonthlyCharges max      : {df['MonthlyCharges'].max():.2f} (should equal cap={cap:.2f})")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14214 entries, 0 to 14213
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   gender            14187 non-null  category
 1   SeniorCitizen     14154 non-null  boolean 
 2   Partner           14153 non-null  boolean 
 3   Dependents        14124 non-null  boolean 
 4   tenure            14114 non-null  Int16   
 5   PhoneService      14095 non-null  boolean 
 6   MultipleLines     14076 non-null  category
 7   InternetService   14053 non-null  category
 8   OnlineSecurity    14023 non-null  category
 9   OnlineBackup      14013 non-null  category
 10  DeviceProtection  14019 non-null  category
 11  TechSupport       14017 non-null  category
 12  StreamingTV       14014 non-null  category
 13  StreamingMovies   14035 non-null  category
 14  Contract          14092 non-null  category
 15  PaperlessBilling  14092 non-null  boolean 
 16  PaymentMethod     1408

## 💾 Save to primary layer

In [16]:
OUTPUT_PATH = Path("../../data/03_primary/Churn/churn_primary.parquet")
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)

df.to_parquet(OUTPUT_PATH, index=False)

print(f"Saved to: {OUTPUT_PATH}")
print(f"File size: {OUTPUT_PATH.stat().st_size / 1024:.1f} KB")

Saved to: ../../data/03_primary/Churn/churn_primary.parquet
File size: 151.0 KB


## 📊 Analysis of Results and Conclusions

- All 17 previously `object` columns were cast to their correct dtypes (bool, category, float64)
- 5 rogue numeric strings in categorical columns replaced with `NaN`
- `TotalCharges` is now `float64`; rows with whitespace become `NaN`
- `MonthlyCharges` extreme outlier (~5 × 10¹²) capped at 99th percentile (114.85)
- `TotalCharges` negative values (data corruption) clipped to 0; upper outliers capped at 99th
  percentile (8045.81) — identified via EDA in `2-exploration/01_jdg_churn_data_description`
- `Contract` carries its natural order (Month-to-month < One year < Two year)
- Memory usage reduced due to category and integer downcasting

## 💡 Proposals and Ideas

- Proceed to `2-exploration/01_jdg_churn_data_description_20260221.ipynb` for EDA on
  `churn_primary.parquet`
- Investigate imputation strategies for the remaining NaNs in feature engineering
- Consider ordinal encoding for `Contract` when feeding to tree-based models